# In Depth: Gaussian Mixture Models

The k-means clustering model explored in the previous section is simple and relatively easy to understand, but its simplicity leads to practical challenges in its application. In particular, the nonprobabilistic nature of k-means and its use of simple distance-from-cluster-center to assign cluster membership leads to poor performance for many real-world situations. In this section we will take a look at Gaussian mixture models, which can be viewed as an extension of the ideas behind k-means, but can also be a powerful tool for estimation beyond simple clustering. We begin with the standard imports:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np

# Motivating GMM: Weaknesses of k-Means

Let’s take a look at some of the weaknesses of k-means and think about how we might improve the cluster model. As we saw in the previous section, given simple, well-separated data, k-means finds suitable clustering results.

For example, if we have simple blobs of data, the k-means algorithm can quickly label those clusters in a way that closely matches what we might do by eye (Figure 5-124):

In [ ]:
# Generate some data
from sklearn.datasets.samples_generator import make_blobs
X, y_true = make_blobs(
    n_samples=400,
    centers=4,
    cluster_std=0.60,
    random_state=0,
)
X = X[:, ::-1] # flip axes for better plotting

In [ ]:
%matplotlib notebook
# Plot the data with k-means labels
from sklearn.cluster import KMeans
kmeans = KMeans(4, random_state=0)
labels = kmeans.fit(X).predict(X)
plt.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis');

> **Figure 5-124: _k-means labels for simple data_**<br><img width="500" src="data:image/png;base64, ..." />

From an intuitive standpoint, we might expect that the clustering assignment for some points is more certain than others; for example, there appears to be a very slight overlap between the two middle clusters, such that we might not have complete confidence in the cluster assignment of points between them. Unfortunately, the k-means model has no intrinsic measure of probability or uncertainty of cluster assignments (although it may be possible to use a bootstrap approach to estimate this uncertainty). For this, we must think about generalizing the model.

One way to think about the k-means model is that it places a circle (or, in higher dimensions, a hyper-sphere) at the center of each cluster, with a radius defined by the most distant point in the cluster. This radius acts as a hard cutoff for cluster assignment within the training set: any point outside this circle is not considered a member of the cluster. We can visualize this cluster model with the following function (Figure 5-125):

In [ ]:
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
def plot_kmeans(kmeans, X, n_clusters=4, rseed=0, ax=None):
    labels = kmeans.fit_predict(X)
    # plot the input data
    ax = ax or plt.gca()
    ax.axis('equal')
    ax.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis', zorder=2)
    # plot the representation of the k-means model
    centers = kmeans.cluster_centers_
    radii = [cdist(X[labels == i], [center]).max()
             for i, center in enumerate(centers)]
    for c, r in zip(centers, radii):
        ax.add_patch(plt.Circle(c, r, fc='#CCCCCC', lw=3, alpha=0.5, zorder=1))

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=0)
plot_kmeans(kmeans, X)

> **Figure 5-125: _The circular clusters implied by the k-means model_**<br><img width="500" src="data:image/png;base64, ..." />

An important observation for k-means is that these cluster models must be circular: k-means has no built-in way of accounting for oblong or elliptical clusters. So, for example, if we take the same data and transform it, the cluster assignments end up becoming muddled (Figure 5-126):

In [ ]:
rng = np.random.RandomState(13)
X_stretched = np.dot(X, rng.randn(2, 2))
kmeans = KMeans(n_clusters=4, random_state=0)
plot_kmeans(kmeans, X_stretched)

> **Figure 5-126: _Poor performance of k-means for noncircular clusters_**<br><img width="500" src="data:image/png;base64, ..." />

By eye, we recognize that these transformed clusters are noncircular, and thus circular clusters would be a poor fit. Nevertheless, k-means is not flexible enough to account for this, and tries to force-fit the data into four circular clusters. This results in a mixing of cluster assignments where the resulting circles overlap: see especially the bottom right of this plot. One might imagine addressing this particular situation by preprocessing the data with PCA (see “In Depth: Principal Component Analysis” on page 433), but in practice there is no guarantee that such a global operation will circularize the individual data.

These two disadvantages of k-means - its lack of flexibility in cluster shape and lack of probabilistic cluster assignment - mean that for many datasets (especially low-dimensional datasets) it may not perform as well as you might hope.

You might imagine addressing these weaknesses by generalizing the k-means model: for example, you could measure uncertainty in cluster assignment by comparing the distances of each point to all cluster centers, rather than focusing on just the closest. You might also imagine allowing the cluster boundaries to be ellipses rather than circles, so as to account for noncircular clusters. It turns out these are two essential components of a different type of clustering model, Gaussian mixture models.

# Generalizing E-M: Gaussian Mixture Models

A Gaussian mixture model (GMM) attempts to find a mixture of multidimensional Gaussian probability distributions that best model any input dataset. In the simplest case, GMMs can be used for finding clusters in the same manner as k-means (Figure 5-127):

In [ ]:
from sklearn.mixture import GMM
gmm = GMM(n_components=4).fit(X)
labels = gmm.predict(X)
plt.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis');

> **Figure 5-127: _Gaussian mixture model labels for the data_**<br><img width="500" src="data:image/png;base64, ..." />

But because GMM contains a probabilistic model under the hood, it is also possible to find probabilistic cluster assignments - in Scikit-Learn we do this using the predict_proba method. This returns a matrix of size [n_samples, n_clusters] that measures the probability that any point belongs to the given cluster:

In [ ]:
probs = gmm.predict_proba(X)
print(probs[:5].round(3))

Output:<br>`
[[   0.       0.       0.475       0.525]
 [   0.       1.       0.          0.   ]
 [   0.       1.       0.          0.   ]
 [   0.       0.       0.          1.   ]
 [   0.       1.       0.          0.   ]]
    `

We can visualize this uncertainty by, for example, making the size of each point proportional to the certainty of its prediction; looking at Figure 5-128, we can see that it is precisely the points at the boundaries between clusters that reflect this uncertainty of cluster assignment:

In [ ]:
size = 50 * probs.max(1) ** 2 # square emphasizes differences
plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis', s=size);

> **Figure 5-128: _GMM probablistic labels: probabilities are shown by the size of points_**<br><img width="500" src="data:image/png;base64, ..." />

Under the hood, a Gaussian mixture model is very similar to k-means: it uses an expectation-maximization approach that qualitatively does the following:

1. Choose starting guesses for the location and shape
2. Repeat until converged:
  1. E-step: for each point, find weights encoding the probability of membership in each cluster
  2. M-step: for each cluster, update its location, normalization, and shape based on all data points, making use of the weights

The result of this is that each cluster is associated not with a hard-edged sphere, but with a smooth Gaussian model. Just as in the k-means expectation-maximization approach, this algorithm can sometimes miss the globally optimal solution, and thus in practice multiple random initializations are used.

Let’s create a function that will help us visualize the locations and shapes of the GMM clusters by drawing ellipses based on the gmm output:

In [ ]:
from matplotlib.patches import Ellipse
def draw_ellipse(position, covariance, ax=None, **kwargs):
    """Draw an ellipse with a given position and covariance"""
    ax = ax or plt.gca()
    # Convert covariance to principal axes
    if covariance.shape == (2, 2):
        U, s, Vt = np.linalg.svd(covariance)
        angle = np.degrees(np.arctan2(U[1, 0], U[0, 0]))
        width, height = 2 * np.sqrt(s)
    else:
        angle = 0
        width, height = 2 * np.sqrt(covariance)
    # Draw the ellipse
    for nsig in range(1, 4):
        ax.add_patch(Ellipse(position, nsig * width, nsig * height,
                             angle, **kwargs))
def plot_gmm(gmm, X, label=True, ax=None):
    ax = ax or plt.gca()
    labels = gmm.fit(X).predict(X)
    if label:
        ax.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis', zorder=2)
    else:
        ax.scatter(X[:, 0], X[:, 1], s=40, zorder=2)
    ax.axis('equal')
    w_factor = 0.2 / gmm.weights_.max()
    for pos, covar, w in zip(gmm.means_, gmm.covars_, gmm.weights_):
        draw_ellipse(pos, covar, alpha=w * w_factor)

With this in place, we can take a look at what the four-component GMM gives us for our initial data (Figure 5-129):

In [ ]:
gmm = GMM(n_components=4, random_state=42)
plot_gmm(gmm, X)

> **Figure 5-129: _Representation of the four-component GMM in the presence of circular clusters_**<br><img width="500" src="data:image/png;base64, ..." />

Similarly, we can use the GMM approach to fit our stretched dataset; allowing for a full covariance, the model will fit even very oblong, stretched-out clusters (Figure 5-130):

In [ ]:
gmm = GMM(n_components=4, covariance_type='full', random_state=42)
plot_gmm(gmm, X_stretched)

> **Figure 5-130: _Representation of the four-component GMM in the presence of noncircular clusters_**<br><img width="500" src="data:image/png;base64, ..." />

This makes clear that GMMs address the two main practical issues with k-means encountered before.

# Choosing the covariance type

If you look at the details of the preceding fits, you will see that the covariance_type option was set differently within each. This hyperparameter controls the degrees of freedom in the shape of each cluster; it is essential to set this carefully for any given problem. The default is covariance_type="diag", which means that the size of the cluster along each dimension can be set independently, with the resulting ellipse constrained to align with the axes. A slightly simpler and faster model is covariance_type="spherical", which constrains the shape of the cluster such that all dimensions are equal. The resulting clustering will have similar characteristics to that of k-means, though it is not entirely equivalent. A more complicated and computationally expensive model (especially as the number of dimensions grows) is to use covariance_type="full", which allows each cluster to be modeled as an ellipse with arbitrary orientation.

We can see a visual representation of these three choices for a single cluster within Figure 5-131:

> **Figure 5-131: _Visualization of GMM covariance types_**<br><img width="500" src="data:image/png;base64, ..." />

# GMM as Density Estimation

Though GMM is often categorized as a clustering algorithm, fundamentally it is an algorithm for density estimation. That is to say, the result of a GMM fit to some data is technically not a clustering model, but a generative probabilistic model describing the distribution of the data.

As an example, consider some data generated from Scikit-Learn’s make_moons function (visualized in Figure 5-132), which we saw in “In Depth: k-Means Clustering” on page 462:

In [ ]:
%matplotlib notebook
from sklearn.datasets import make_moons
Xmoon, ymoon = make_moons(200, noise=.05, random_state=0)
plt.scatter(Xmoon[:, 0], Xmoon[:, 1]);

> **Figure 5-132: _GMM applied to clusters with nonlinear boundaries_**<br><img width="500" src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAe4AAAFVCAIAAACmeJu3AAAACXBIWXMAAAsSAAALEgHS3X78AAAgAElEQVR42u3dfVBU99338e8qgoTFAgZvQ5NZKVOu1jZNR5OZMB1FzUgSpT4UUNFCGp1tbhvTVJtgsTNpUmPIxZiZZKo4xsRqvK4Em0S6qSmpTaTMPY5OKbmwgUykF8Gd2LUpym4EJK7ouf9YXVfYh7OPnHP2/foLFxaOv93zOd/fw/mtSVEUAQDo2QSaAACIcgAAUQ4AIMoBgCgHABDlAAAdRfnJkyerqqp8H9m3b19paWl1dXV1dfXp06dpUABIvBT1P/rKK6/YbLaMjAzfB7u6uurr62fOnElTAoAOqnKLxbJz585RD3Z1de3evXv16tUvv/wyrQkAWo/yhQsXTpw4cdSDixcvfuaZZ1577bX29vbW1lYaFAA0HeV+PfTQQ1lZWSkpKcXFxR9//HGgH2N7AACIn5Rwn+AbyoODg6Wlpc3NzZMnTz5x4kR5eXmgZ5lMpr6+Ae03R25uJsdJY3KcHKd2DjJeUW4ymUTk8OHDw8PDFRUVmzZtqqqqSktLKyoqmjt3LtdGAEg8U8KGPrhQU5VznBwnxxmnqpxbhABA94hyACDKAQBEOQCAKAcAohwAQJQDAIhyAABRDgBEOQCAKAcAEOUAAKIcAIhyAABRDgAgygEARDkAEOUAAKIcAECUAwCIcgAgygEARDkAgCgHABDlAECUAwCIcgAAUQ4AIMoBgCgHABDlAACiHABAlAMAUQ4AIMoBAEQ5AIAoBwCiHABAlAMAiHIAAFEOAEQ5AIAoBwAQ5QAAohwAiHIAgI6k0ASIq/5+1+bNLXb7FIvli/r6BdnZWbQJQJRDN8G9d+9SkYmbN7fYbFUipo4OReTAnj3LaSWAKIem+Qb3+vWNO3aU2u1TREwiImKy26fQREA8MFaOWPIN7t5es4hYLF+IKCIiolgsF2gigKocWmexfNHRoYiYRJT8/EERqa9fIHLAbp9isVyor59PEwFEObTON7h37Vpy5YpkZ2cxPg4Q5YhckNUjcVpY4hvcOTmZfX0DwiIWgChHNIKsHlG/sCT6II7VIhYuCUBsovzkyZPbt28/cOCA95GjR482NDSkpKSUlZVVVFTQoJoSZPWI+oUl0QdxkL8VVjqzrhGIQZS/8sorNpstIyPD+8jIyMjzzz9/6NChtLS0ysrK++67LycnhzbVDt9JyFGrR4J8S30QR38YYaUz6xqBGES5xWLZuXNnTU2N95Genh6LxWI2m0Vk9uzZbW1t999/P22qHUFWjwT61tgyWX3oj3pubm5myMMIq2BXfyRA0lHCcebMmZUrV3r/+be//W3jxo2er1966aU333xTgcacO+dcseL1e+55Z8WK/z5/3hny51eseF3kqogicnXFitcVRTl/3vsbXg/+G8Y+V8Wf++9AT4nmSIBkE9W0p9lsHhwc9Hw9NDQ0ZUqwPq9nMYPG5eZmGuw4rdZ3PCMYbW3KpUuhx5e7u9O9ZXJ3d3pf34DIxB07Sj3fvXIl2Ovo+9wjR4b7+11XrkwMPhq+deucS5euFexbt873/eVhHUk0M6LGe9E5TsMcp7drG8sBFm8V7/26oKDAbrdfuHBh8uTJbW1t69ato5ejNeGOL/sOYtx227+s1ib1+ej7XJdr8vr1zTt2lAYfDQ+y6jys4RRmRMFYeXhMJpOIHD58eHh4uKKiora2du3atYqiVFRUTJs2jQZNGJV1qG8gnj79D6s1RCj7jmu73Slh5WN9/YLW1u0u1zdEBkUW9fb+P4lirjKs20SZEQVRHoavfvWrjY2NIlJaeq2fO2/evHnz5tGOiReyDvVkfU9PSl5e3eDg9AsXJrtcVpvtK8FD2bdMLin5IKx8zM7OKi6eZrOV+t64H/FcZVi3iTIjCqIcxhw58Wa9iJKV9VuR1T5PjFc+jr1xP1Z7sATvhaj5K4m/9xUgyhFtzvpmvcg5kbCL1ghSeOyN+7HagyVIL0RlEMfk3leAKEcshcxZ36wvKspMTQ27NNbUTlhBeiEqx5qOHBGRN0QWiWRFfO8rQJQjlkLm7M1ZX6r3QYMgvZCwxppEGkVWRXzvK0CUQ1tZb5heSFhjTenpl0tKDqi89xUgyoEEXZk8QdzTM7G/3/7pp4VW66FRI+a+WV9SImN/D5uqgygHohVy3jL4D3iC2Gpt6uysdThMH300esScohtEOQjZuK/SCzlvqWaFie8oSmvriNPp8h6tyqKbJYkgymHMwE3MKr2Q85ZqVpiM2jagpqYl3KNlSSL0awJNYACeDOroWGazVdfUtCQyZGPCYvlCxLO3j/95y5A/ICL19QuysraL/EGkUWRRBEfLkkRQlWM8i+74ZVBiVumFHMtWM9g9atuACI6WJYkgyhH3ojtIx3/69D7vzZy33XYukSEbEyHHslUOdqs/Wt+r4969S0UmCrOjIMoRVyGLbpNpROQNkUyRAZHLiQxZTVF/tL5Xx/XrGz3boLMkEUQ54ihkx//s2dtEll3/+veR/ZWkWr/he3Xs7TUHb5aNG989fnxAZGpR0ZUXX3yAlS0gyhGXUY5AWc/n2au5Ono24w3SLM3NU0QeETE1NyupqaxsAVGeHGJe3oa53cr8yNI5qdZvjN2MN2izTGRlC4jypJP48jZQ1oeVzkm1fmPsZrxBmyUlgi2CAaJc36Isb2NY1IeVzqzfCNQsbvfh48e3i0wtKrpaX38/bQKiPCkECVA1MR3Doj6sdGb9RqBm2b//h7QDiPJkrOMCBajfmB6V7zEcsyadAaIcsQ9QvzE9Kt8tFoV7DrWDPbZAlGM0v2Mvo/L94MHZjFlrh++F1u3enZp6C7EOojzZ+f2QhFH5PqqopyocX74X2uPHB1yuR9g6EUR5svP7IQnBJyfZeVU7HSmRqSwwB1EOP4We3T4l+OQkO69qoSPludC63Veam5nGAFGOMYVeyERg51UtdKQ8XzudrtRUpjFAlGNMoRcyEbhzJ5HUfIgorQSiHGEnAvGRSGNnJph2BlEO6MzYmQmmnaELfLYncMPYDxEdFe79/S6rtamk5AOr9ZDT6aLFQFUOaM7YmYlR084U6SDKcQMjsNo0dmZiVLivXNnO2lAQ5QT0NRR3Og131oaCKDc+9QHNjT86xdpQEOXGpz6gKe6MUaRH3z8DiHLNUR/QFHd6EUEo9/e7Fiw44HB8W2Swo2OJyB8YQANRboTe99g44MYfvYhgVmPz5haHo/b6DlyNDKCBKDdI4cYkp35FMKvh+xSRDIuF5ecgyg1RuDHJqV8RzGr4PiUvr7O+vopmBFFuhMKNSU79imBW4+anVDHnCaLcIIUbk5z6FcGsBhMhIMqNWbhxbgMgynVcuLG+GABRrnusXQGQAGxyG1+sXQFAlOve2P2vASDmGGCJL++M6PTp59zuyyUlHzBoDoAo1xnvjKjV2mSzrWPQHEA8MMCSIAyaAyDKdY9BcwDxwwBLgnDDZzLze3sB9xyAKNfEqRgWbvhMZn5vL/jZz/703nvXpk/c7lf3719JQyFiDLCEcSp2dCyz2apralq8+W61NpWUfGC1HnI6XX4vAMF/QHfXMyP9dxLJd6aktXXE03onTkzwPvinP12hVZGIqlxRlKeffvrUqVOpqanbtm274447PI/v27fvrbfeysnJEZFf//rXM2bMMPyp6J20DHknp8Fu9eTO1Yh7b74brrlck2tqWvbsWS5yTuTag1evDthsP6ZVEfcof//9991ud2Nj48mTJ+vq6hoaGjyPd3V11dfXz5w509jNdP1U/ELkj6dPf2m1HqqvXxByUYrBVq2wCCfiq119/YLW1u0u1zdEBkUW2e1/EZGiInNz8xsimSIDIlm0aoKvtdu3z3viiVbDzFWojfL29vY5c+aIyF133dXZ2el9vKura/fu3X19ffPmzfvxj39s1HeAZ9KytfVzl+sJl8tksykiBywWJfjetgbbppxd1yO+2mVnZxUXT7PZSn1b78UXv5+a2tLa2udyTRYpp1UTfK1tb//PM2c2G6aXqTbKBwcHMzMzrz0nJeXq1asTJkwQkcWLF69Zs8ZsNj/66KOtra3FxcWGfAd4Ji1LSj7o6Lhxoh48ODv4ohSDrVphEU40V7uxred5UzmdrpqaFrv9L7Rqgq+1/f15Ruplqo1ys9k8NDTk+dqb4yLy0EMPmc1mESkuLv7444+DRHlubqYuWiTIcRYWXvSeqIWFw4WFd/z+99XBf1XwH4jTccbvL4b73zHAix6WvXuXrl/f2Ntrzs8f3LVrSU5OZsjWC6tVk609Y873FM7J+efFizdOZ720bbRRPmvWrJaWlgceeKCjo6OwsNBbqpeWljY3N0+ePPnEiRPl5eVBfkNf34CWG8IziOZwZOfl9QcaONu6dc6lS9cKq61b54/j/yg3N1Oz7ekdjiwsvLh16xztD0HGtDEn7thR6vnqypUYv+e1/KLr5Th9T+EXXlj9859r4nSOyUXRpCiKmp/zrmARkbq6uq6uruHh4YqKinfeeee1115LS0srKirasGGDfqPcam3yDKKJKEuXan3gTMtni75akojkOI3Rv1FblZtMpmeeecb7z/z8fM8XS5YsWbJkieif3wkr7seLVUtibK+FNxXGYYDF8PxOWLGSOlYtCS/eVBq5mgYfTSXK9cqzwMDhyM7Lc3oXElBgRtySdvuUwsLhrVtZkkGvRbtXUxHjXE2J8ms8K8NGDZ9RYEbckqKfMVN6LVxNifJkKTBZ8xu8r8rIL28qrqZEuQ4KTITsqzLyy5tKX1fTUaOpRLmhCswNGw53d6dTYCZ5X5UujrFbzO9oKlGepAUmZ52B+6p0cWgxotxoBWagyOY95NtXZeQ3JnVlT08KXZx4dwp1dzcyUR6bAjNQZDOw4NtX5c0Tk7oyL6/Ou4M5XZw4dQp9G/zSJeMUYUT5TQVmWlpjd3e6p8D0XL2PHBG/kc3AQrJVPQmoK7/ylQKROqfz9uzsz7ZsWUrjxKNTaNQijCi/qcA8eLDSOxNyfS+RN7yF0vTp56zWJk9feMuW2QwsJFXVk4C6sre388svfy1iGh5WnnvuwJ49Fton5p1Cvx8jY4CCgygPefVeJNKYnn65pETc7ss22zrGx4MX46dPjzD0FFZd2dZW53B8W2Toyy+/SdMlppAf9TEyBjiX+ZjmYFdvEUUkS2RVSYns2bP87NnbONOCF+MdHctcrjQRz3abDD2pqiunTZspskSk0tNoo5rO++nYK1e+zuc4x6qQnzHj6wY7l6nKQ1y9fYdQGB9X0YkRkcVZWdtnzPg6e7CEP8byYF5e3bRpM31H7RiwinObG+RcJspDXL2DhztuPjecIu+JZNxyi/vgwdmFhXewB0v4RUPVqHFb1krFqc191zgQ5cke7vA9N9radjkctSImh+P7NTUH4vdheEn1vqIvGKc2913jQJQDN86NadNmOhzUj/Gq2RmwAlEeA9ypHxL1Y1xrdjYNju0prPfPZSbKI8Sd+urrR+YSoP1T2GADgES5Wsw+qa8f6dyAU5goZ/SAzg3AKUyUM3owfnyL7r17l4pMpHMDTmGiXGejB/Atutevb9yxo5TOTcTXQu8AlCE/IX58m9R4iHLEkm/R3dtrpnMTzbXQOwBlyE+IH5cmvXTp1bS01CC9RqIc8C26vxD54//+r3vstnN0btRfC70DUIxKxapJT5yY4HIF6zXqV7Jvp+Xdq8hqPcReRdGrr1+wdOmBrKw9IpVO549stuqamhaaJaxr4dgdtfw+CJWns2/riZwP3mukKtfTi+07cGbspaaJ5ym6S0o+6Oi4dsK0to6UlHzA0kP118KxA1CG/IT4eI+leEeifJvU7c5obr42VZOfP0iUG+fFpusaz2EWk4jick3u6FjG0sOwroV+H+Ruz7DGUryns2+TOp2u1NRrsb5r15IrV4hyo7zYLKiIX2npcGT/4x+fuFxWrpRIfA3h93T2jfWcnEy20zLOi+0JnZ6eif399k8/LVy58nU+jjJWpWVubuayZf0221dEhCtlBPr7XRs3vnv8+IDI1KKiK//1XxVGWnERvxrCbp8yffo5t/tyUo3smRRFScxf0sgF0Ol01dS0eMciPa/x9Y/xNIkoS5fqYBxAF33t3NzM7u7PxrY2jamS1dpks6WIVHremStW6GPFhRbaU80ZrZeTiKo8WME46kFGzOPa2p6p5pUr25n8DIvdPkVkoved2d0t3k8JpxlVNF1yndF8tqcIi73ibPPmFpvt+x0dwzZb9vz5r7HoM5y35QXvO/PcObvn01NZ4skZTVXuH7v7x79Eek9klfcDhljKovJt6XYfPn58u8jUoqKr//rXf5w5Q98xjDPaOwc29lY1otzI4wDC7v5xK5E6OrIZworgbbl//w+9/9yw4Q//8z+stgrjjLZamzo7ax0O00cfGX8tLFEeejM/RF8itbW95nB8nxiKxq5diy5dYvuacLuDyVJAEOWhN/ND9CVSS0t1TQ0xFJWcHLaviaA7mCz9GKI89GZ+iFWHl3ZAgruDybMNJytYbprsNti2DECSFxD/+Z/zLZYLdvuUmpqjxl46RVV+06XbYNsyAEkueT6AkCg38rYMQJLzHT7t6bnF9x4r9TdSEuUAMJ58Zz77+7s7O2uNuqM1UX5Df79rw4bD3d3p3BgNGIPv8GlPj8XhMOzaRKL8huQZVoMuCotk+HDhePMdPrVaD3V2GnZtIlF+A5tqgcIiSSp0461NJMpv4GMoQGGRJBW68RDlN12009Iau7vTuSORQQMKCxDlOr5oHzxYyWLExAwaXLr0alpaKrGufjTg/HmX1foOLQainHJSQ4MGJ05McLkYCw5jNOAnP2lm9BxEeczKSc6imAwaiJwfNRbsuV729NzS338qJ2dGQcEIV01fvb1mRs9BlMesnOQsismggdud0dx801iw93opojgcjZ2d1Ul11fTt+dXWzq6r+3BULzA/f6CtjdHzyHvPo77L3Z7JXk5yFsVk0MDpdKWm3hgL7u93tbZ+LnJYZEBkkYg52a6avj2/trY6h6N2VC+Q/cqj7D2P+i53exr5uq2mnOQsinmsi4jV2uRyPXF97OWNJLxq+vb8nM7bx/YC2a88yt6zsfvWaqNcUZSnn3761KlTqamp27Ztu+OOOzyPHz16tKGhISUlpaysrKKiQu/XbZW5g7iehCkpA9/4hrmg4EBSXTV9e37Z2WeGh+kFxrj3bOy+tdoof//9991ud2Nj48mTJ+vq6hoaGkRkZGTk+eefP3ToUFpaWmVl5X333ZeTk6Pr6za0cBIuXpy+Z8+SKDtSuuPb89uyZclzz9ELjKoNa2tn+e6DmJ2dxd2eIiLt7e1z5swRkbvuuquzs9PzYE9Pj8ViMZvNIjJ79uy2trb7779f19dtaOEkHHWaJcnyoVE9vz17LLwromlDq7Vp1NuGuz1FRAYHBzMzr034pqSkXL16dcKECb4PZmRkDAwM6DoyoIWTsL/fVVNzUw1ORwr0v2MW5WazeWhoyPO1J8c9Dw4OXvsEtaGhoSlTgrWXRpb+5OZmBp+51ssSJV0cZ5CDPH/e9ZOfNPf2mvPzB3btWpSTc23YZMOGw95iKi2t8eDBysLCi96OVGHhcDz+47zoxjtONW8bI61HVBvls2bNamlpeeCBBzo6OgoLCz0PFhQU2O32CxcuTJ48ua2tbd26dUF+gy5uiM/NzeQ4E3OQVus7nshua1MuXboxbNLdne4tprq70/v6BrZuneNdhLd16/yY/8f19aJrf+ZAI+0Z8m2jl5MoxlG+cOHCY8eOrVq1SkTq6uoOHz48PDxcUVFRW1u7du1aRVEqKiqmTZtGtw5R9n/HTmawfMgXNx6rlGxvG7VRbjKZnnnmGe8/8/PzPV/Mmzdv3rx5vG8QrkDzz0xmRHYJhCT3Xknc7YnxESiyqcEjuwQiybssRDno/xrhEogk77IQ5QCXQLosRLnOGXuzNKO+TOx8C7osRPlNjL1ZmlFfJspS0GUhym/CegBdVN+8TPS6QJQHw3oAXVTfvEz0urgMEOXBsB5AU6WZ76dP+FbfvEx0juNxGSDKjYP1AJoqzXw/fcJiGeFlonMcsvpm8I0oh3ZLs6ysL+vrF9ImdI69fvazP7333jpP9e12v7p//0r1lwGiHBif0qy4OIW5LzrHvk6cmOC90h87NsH7sRJbtsxm8I0oh85KMySxcyLXrvTDw2dtti2MjxPl0GtphqRVVGRubn5DJFNkID39yuXLjI8T5dAbFpzhxRe/n5raYrdfsVhG3O5bm5sZHyfKoTcsOINvp83pdKWmMhZHlENvWHBGt2xUt4zLOVEO/Qm54IwRGJ0mMt0yohxJJOTiFk51XYj4ZaJbRpTDCIJ3qPv7Xa2tI5zq2hdxInMfEFGOpKj1XK4076JjTnXNijiRueeAKEeS1HrzRBpFzFlZn9TXr6JNtCniRGaeM6QJNAEMUOuJfEWkUqS0uPj/MOepWZ5EbmycLSIrV7ZbrYecThfNQlUOqK31WOKiHSonP3nJiHIYUJATW03vmyUu2qFy8pOXjChH8pZyUcYHEkDl5CcvGVGO5C3lAtXy//53l0ixSDZLXMadyslPFiAS5UjeUi5QLS+yNC+vbtq0maxmG3cql6OwAJEoR/KWckFq+WnTZh45ch8tabDEB1EO45/YdNKNhDUtRDmo5emk6x5rWohyJGnVRifdSFjTQpQjKXJ8wYIDDkctVZtRMVxGlCMpet8Ox7ep2gyM4TKiHEnS+x5kf0QDY7iMKEeS9L6XiDSKZOTlddbXV9EmCeM7S7F371KRibQJUQ5E3Pv+g90+xWJx1ddXsVItkXzXlqxf37hjRyltQpQD9L51xndtSW+veWypzjLweGO/ciDgoIHV2lRS8gHbaodksXwhooiIiJKfP+h5cOPGd222lI6OYZvNcc89fwrejLQ2VTkQ90EDljYG57u2ZNeuJVeuiIgcPz4g8ohIo8gvLlww2WzBmpHWJsqBgIVeoA6+mmk6bkhRz3d0Kycns69vQEREpoqYRMxqmpHWjhIDLDB4Wd3Rscxmq66paQn0rfXrm0MOGqhZ2qjfIYIojzzQ04uKrogoIgNqmjHc1gZVOZJFkELP7zRdkEEDNTek6HeIIJojH3uT7e9/X+351osvPpCaeqCnZ2J/f93UqYVf+9pFTzP67S1x+w9RDgQs9ALd5+37Le80XZBBgyivHHq85qlcfxLkJttADej3ysECJKIckHDLar/TdKOKzc2bW3p6bunvP5WTM6OgYCTkcjr97hDi98hVluoR3GTLsDhRDoQhSKEXYJrOT+UoojgcjZ2d1SFHHrQ2RDC2rO7psZeVveN03p6d/VlT09L8fEuQI1cZuBHcZMuuWEQ5MA5jDp41GCHrR60NEYwtq8vK3vEMag8PK8uX13V0PBbkyFUGbgQ32TIsTpQDCapk//3vLpGlniDzDCAEijPN3tM4tqx2Om/3PuJ03h6TwI3gAsawOFEOJKiSFSnOy6ubOrXw/PnunBxLQcGBQHGmckw58Yk/tqzOzv5sePjaI9nZZwhcohww/LhK9vWPdb4/3OI3msSPobFldVPT0uXL65zO27OzzzQ1LeHlJsoBY4pgRk7lU9Qv24hV/T62rM7Pt3jHx0NiMyyiHNCrCGbkVD5lVOIHCUqN3GrEpihEOaBdwYtNNQPEY3+Dmpgblfg1NQGD0rd+7+m5xWptCvmRDvGooFn9TZQD2hV9sRnZbxiV+EGC0rd+7+/v7uysDfmRDuEe0vUboCb299unTi382teGxl4AWP1NlAPaFX2xGZNydfr0Pu8dkrfddi5Q/d7TY3E4QuwVE8EhXY/+RpFah8P00Ud+LgCs/jZmlF+6dOnJJ588f/682Wx+/vnns7Ozvd/atm3bhx9+mJGRISINDQ1ms5lmhWZFX2zGpFw1mUZE3hDJFBn48EN7SckH3rER3/rdaj3U2Rlir5gIDul69AfbfpbFiMaM8jfeeKOwsHDDhg1//OMfGxoafvnLX3q/1dXV9eqrr2ZlMcENHYim2PSMS3z6aUZe3nM5OTMKCq5EXK6ePXubyDLP159//s7nny/xOzbid6+YsSPj4f6nrkf/QFh7p8AIUd7e3m61WkVk7ty5DQ0N3scVRbHb7U899VRfX195eXlZWRltCi2Lptj03ZjlnnuiWtThW0eLDKkpjb17xYwdGff8mCfiV65sDzn56Yn+sdvPhnVJY52iPqL8rbfe2r9/v/eft956q2fkJCMjY3DwRkfv4sWLVVVVDz/88MjISHV19Z133llYWOj3F+bmZuqiRThOGjMQhyPbOyLhcGRH87/bu3fp+vWNvb3ms2c7z5z5v566qLBwOPjv9Hw30GFs2HDYG/FpaY0HD1YG+T3eXcUjEPIPcRJpKMrLy8vLy8u9/3zssceGhoZEZGhoKDPzxv8/PT29qqoqLS0tLS3t3nvv/eSTTwJF+djN57T50nKcNGYgeXn93hGJvDxnkGepqFsnepajOJ3franxbEd1YevW+UF+p/c4Ax1Gd3e6N+K7u9Pj1/jB/xAn0bhcbNQOsMyaNau1tfXOO+9sbW29++67vY/39vZu3LjRZrONjIy0t7f/4Ac/oKcDo1I/JK1+dWAEAz6BDiNhywdZp6inqnyUysrKzZs3r169OjU19YUXXhCRffv2WSyW+fPnL1u2rKKiYtKkScuXLy8oKKBNYVTqYzeu99cEOoza2tltbde2WNmyJY5brLBOUcdRPnny5Jdeesn3kR/96EeeL9auXbt27VqaEhjfurWu7kPvduTPPXdgzx7LuF/SoLkoB6DxupVb7YlyALqvWxnCJsoB0BUAUQ4gFqK5+ybKrgA3/hDlAGJjHHcJZ4NyXZtAEwDaMY5Tl8yaEuUAYsNi+UJEEZHET12O459G9BhgATRkHKcumTUlygHExjjefcONP7rGAAsAEOUAAKIcAECUAwBRDhSUHTYAAAWgSURBVAAgygEARDkAgCgHAKIcAECUAwCIcgAAUQ4ARDkAgCgHABDlAACiHACIcgAAUQ4AIMoBAEQ5ABDlAACiHABAlAMAiHIAIMoBAEQ5AIAoBwAQ5QBAlAMAiHIAAFEOACDKAYAoBwAQ5QAAohwAQJQDAFEOACDKAQBEOQCAKAcAohwAQJQDAIhyAABRDgBEeSB//vOff/7zn4968He/+11ZWdmqVav+8pe/0KAAkHgp6n9027Ztx44d++Y3v+n74Llz5w4cONDU1PTll19WVlZ+73vfmzRpEs0KABqtymfNmvX000+PevDvf//77NmzU1JSzGbzjBkzTp06RZsCgFaq8rfeemv//v3ef9bV1T344IN//etfR/3Y4OBgZmam5+tbbrllYGCANgUArUR5eXl5eXl5yOebzebBwUHP10NDQ1OmTAn0k7m5mbpoEY6TxuQ4OU7diXYFy3e+85329na32z0wMPDpp59+/etf5/IIAFqpykPat2+fxWKZP39+VVXV6tWrFUXZtGlTamoqbQoACWZSFIVWAABd4xYhACDKAQBEOQCAKAcAojzO/G7bsm3btrKysurq6urqau+y9HGk/b1lLl269NOf/nTNmjWPPPKI0+nUYGMqivKrX/1q1apV1dXVn332mffxo0ePlpeXr1q16s0339TCOz7Qce7bt6+0tNTTjKdPn9bCoZ48ebKqqsr3Ea01ZqDj1FpjjoyM1NTUrFmzZsWKFUePHtVsewY6TlXtqcTTs88+++CDD27atGnU45WVlU6nU9EGvwfZ19dXWlp6+fLlgYGB0tJSt9s9vgf529/+9je/+Y2iKO++++6zzz6rwcY8cuTIL37xC0VROjo61q9f73nw8uXLCxcuHBgYcLvdZWVl58+f1+ZxKoryxBNPdHV1KZqxZ8+e0tLSlStXeh/RYGP6PU4NNubbb7/93HPPKYricrnmzZun2fb0e5wq2zO+VbnfbVsURbHb7U899VRlZeXbb7897ldCXewt097ePnfuXBGZO3fu8ePHNdiY7e3tc+bMEZG77rqrs7PT82BPT4/FYjGbzZMmTZo9e3ZbW9u4v9x+j1NEurq6du/evXr16pdfflkLBZrFYtm5c6fvIxpsTL/HqcHGfPDBBx9//HERuXr1akpKimbb0+9xqmzPlBgeh8ptWy5evFhVVfXwww+PjIxUV1ffeeedhYWFCWssvewtM+o4b731VrPZLCIZGRm+oyjj25iBWiwlJeXq1asTJkzwfTAjI0MLW/T4PU4RWbx48Zo1a8xm86OPPtra2lpcXDy+x7lw4cJ//vOfgY5cI43p9zg12Jjp6emeBnz88cc3btyo2fb0e5wq2zOWUa5y25b09PSqqqq0tLS0tLR77733k08+SWT6xHxvmcQc52OPPTY0NOQ5GO/7b9wbc1SLeY7QU1N48nHcm1HlcYrIQw895LlYFhcXf/zxx+OePhp8T6qnwcY8e/bshg0bfvjDHy5atEjL7Tn2OFW25zisYOnt7a2srPSMVbW3t3/rW9/S4HtRa3vLzJo1q7W1VURaW1vvvvtuDTam9wg7Ojq8l5OCggK73X7hwgW3293W1vbd735XC+NpY49zcHCwtLR0eHhYUZQTJ05o5z3pezO2BhvT73FqsDHPnTu3bt26J598cvny5VpuT7/HqbI9UxJ5oN5tW5YtW1ZRUTFp0qTly5cXFBRoKsS1ubdMZWXl5s2bV69enZqa+sILL2iwMRcuXHjs2LFVq1Z5hq0OHz48PDxcUVFRW1u7du1aRVEqKiqmTZumhQEBv8e5adMmT/+mqKjIMy2hBSaTSUQ025iBjlNrjbl79+4LFy40NDTs3LnTZDKtWLFCm+0Z6DjVtCd7sACA7nGLEAAQ5QAAohwAQJQDAFEOACDKAQBEOQAgKv8fHpHvGJSLjCIAAAAASUVORK5CYII=" />

If we try to fit this to a two-component GMM viewed as a clustering model, the results are not particularly useful (Figure 5-133):

In [ ]:
gmm2 = GMM(n_components=2, covariance_type='full', random_state=0)
plot_gmm(gmm2, Xmoon)

> **Figure 5-133: _Two component GMM fit to nonlinear clusters_**<br><img width="500" src="data:image/png;base64, ..." />

But if we instead use many more components and ignore the cluster labels, we find a fit that is much closer to the input data (Figure 5-134):

In [ ]:
gmm16 = GMM(n_components=16, covariance_type='full', random_state=0)
plot_gmm(gmm16, Xmoon, label=False)

> **Figure 5-134: _Using many GMM clusters to model the distribution of points_**<br><img width="500" src="data:image/png;base64, ..." />

Here the mixture of 16 Gaussians serves not to find separated clusters of data, but rather to model the overall distribution of the input data. This is a generative model of the distribution, meaning that the GMM gives us the recipe to generate new random data distributed similarly to our input. For example, here are 400 new points drawn from this 16-component GMM fit to our original data (Figure 5-135):

In [ ]:
Xnew = gmm16.sample(400, random_state=42)
plt.scatter(Xnew[:, 0], Xnew[:, 1]);

> **Figure 5-135: _New data drawn from the 16-component GMM_**<br><img width="500" src="data:image/png;base64, ..." />

GMM is convenient as a flexible means of modeling an arbitrary multidimensional distribution of data.

# How many components?

The fact that GMM is a generative model gives us a natural means of determining the optimal number of components for a given dataset. A generative model is inherently a probability distribution for the dataset, and so we can simply evaluate the likelihood of the data under the model, using cross-validation to avoid overfitting. Another means of correcting for overfitting is to adjust the model likelihoods using some analytic criterion such as the Akaike information criterion (AIC) or the Bayesian information criterion (BIC). Scikit-Learn’s GMM estimator actually includes built-in methods that compute both of these, and so it is very easy to operate on this approach.

Let’s look at the AIC and BIC as a function as the number of GMM components for our moon dataset (Figure 5-136):

In [ ]:
%matplotlib notebook
n_components = np.arange(1, 21)
models = [GMM(n, covariance_type='full', random_state=0).fit(Xmoon)
          for n in n_components]
plt.plot(n_components, [m.bic(Xmoon) for m in models], label='BIC')
plt.plot(n_components, [m.aic(Xmoon) for m in models], label='AIC')
plt.legend(loc='best')
plt.xlabel('n_components');

> **Figure 5-136: _Visualization of AIC and BIC for choosing the number of GMM components_**<br><img width="500" src="data:image/png;base64, ..." />

The optimal number of clusters is the value that minimizes the AIC or BIC, depending on which approximation we wish to use. The AIC tells us that our choice of 16 components was probably too many: around 8-12 components would have been a better choice. As is typical with this sort of problem, the BIC recommends a simpler model.

Notice the important point: this choice of number of components measures how well GMM works as a density estimator, not how well it works as a clustering algorithm. I’d encourage you to think of GMM primarily as a density estimator, and use it for clustering only when warranted within simple datasets.

# Example: GMM for Generating New Data

We just saw a simple example of using GMM as a generative model of data in order to create new samples from the distribution defined by the input data. Here we will run with this idea and generate new handwritten digits from the standard digits corpus that we have used before.

To start with, let’s load the digits data using Scikit-Learn’s data tools:

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
digits.data.shape

Output:<br>`
(1797, 64)
    `

Next let’s plot the first 100 of these to recall exactly what we’re looking at (Figure 5-137):

In [ ]:
%matplotlib notebook
def plot_digits(data):
    fig, ax = plt.subplots(10, 10, figsize=(8, 8),
                           subplot_kw=dict(xticks=[], yticks=[]))
    fig.subplots_adjust(hspace=0.05, wspace=0.05)
    for i, axi in enumerate(ax.flat):
        im = axi.imshow(data[i].reshape(8, 8), cmap='binary')
        im.set_clim(0, 16)
plot_digits(digits.data)

We have nearly 1,800 digits in 64 dimensions, and we can build a GMM on top of these to generate more. GMMs can have difficulty converging in such a high dimensional space, so we will start with an invertible dimensionality reduction algorithm on the data. Here we will use a straightforward PCA, asking it to preserve 99% of the variance in the projected data:

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(0.99, whiten=True)
data = pca.fit_transform(digits.data)
data.shape

Output:<br>`
(1797, 41)
    `

> **Figure 5-137: _Handwritten digits input_**<br><img width="500" src="data:image/png;base64, ..." />

The result is 41 dimensions, a reduction of nearly 1/3 with almost no information loss. Given this projected data, let’s use the AIC to get a gauge for the number of GMM components we should use (Figure 5-138):

In [ ]:
n_components = np.arange(50, 210, 10)
models = [GMM(n, covariance_type='full', random_state=0)
          for n in n_components]
aics = [model.fit(data).aic(data) for model in models]
plt.plot(n_components, aics);

It appears that around 110 components minimizes the AIC; we will use this model. Let’s quickly fit this to the data and confirm that it has converged:

In [ ]:
gmm = GMM(110, covariance_type='full', random_state=0)
gmm.fit(data)
print(gmm.converged_)

Output:<br>`
True
    `

Now we can draw samples of 100 new points within this 41-dimensional projected space, using the GMM as a generative model:

In [ ]:
data_new = gmm.sample(100, random_state=0)
data_new.shape

Output:<br>`
(100, 41)
    `

> **Figure 5-138: _AIC curve for choosing the appropriate number of GMM components_**<br><img width="500" src="data:image/png;base64, ..." />

Finally, we can use the inverse transform of the PCA object to construct the new digits (Figure 5-139):

In [ ]:
digits_new = pca.inverse_transform(data_new)
plot_digits(digits_new)

> **Figure 5-139: _“New” digits randomly drawn from the underlying model of the GMM estimator_**<br><img width="500" src="data:image/png;base64, ..." />

The results for the most part look like plausible digits from the dataset!

Consider what we’ve done here: given a sampling of handwritten digits, we have modeled the distribution of that data in such a way that we can generate brand new samples of digits from the data: these are “handwritten digits” that do not individually appear in the original dataset, but rather capture the general features of the input data as modeled by the mixture model. Such a generative model of digits can prove very useful as a component of a Bayesian generative classifier, as we shall see in the next section.